In [4]:
from keras.utils import Sequence
import numpy as np
import librosa
import os
import IPython.display as ipd

In [2]:
class Datagen:
    def __init__(self, X, y, batch_size, p_white_noise=0, white_noise_strength=0.1, p_cutout=0, n_cutout=0, p_noise=0, 
                 noises=None):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.p_white_noise = p_white_noise
        self.white_noise_strength = white_noise_strength
        self.p_cutout = p_cutout
        self.n_cutout = n_cutout
        self.p_noise = p_noise
        self.noises = noises
        self.n = 0
        self.max = self.__len__()
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(self.X.shape[0])
        np.random.shuffle(self.indexes)
        
    def __data_generation(self, selected):
        X = self.X[selected]
        y = self.y[selected]
        
        X = self.add_white_noise(X)
        X = self.add_cutout(X)
        if self.noises is not None:
            X = self.add_noise(X)
        
        return X, y
    
    def __len__(self):
        "Batches per epoch"
        return int(np.floor(self.X.shape[0] / self.batch_size))
    
    def __getitem__(self, index):
        selected = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(selected)
        
        return X, y
    
    def __next__(self):
        if self.n >= self.max:
            self.n = 0
        result = self.__getitem__(self.n)
        self.n += 1
        return result
    
    def add_white_noise(self, X):
        for i in range(X.shape[0]):
            noise = (np.random.random(size=(X.shape[1], X.shape[2])) * self.white_noise_strength).astype(np.float16)
            if np.random.choice([1, 0], p=[self.p_white_noise, 1-self.p_white_noise]):
                X[i, :, :] = (X[i, :, :]).astype(np.float16) + noise
        return X
    
    def add_cutout(self, X):
        for i in range(X.shape[0]):
            index1 = np.random.choice(np.arange(X.shape[1]), size=self.n_cutout)
            index2 = np.random.choice(np.arange(X.shape[2]), size=self.n_cutout)
            if np.random.choice([1, 0], p=[self.p_cutout, 1-self.p_cutout]):
                X[i, index1, index2] = 0
        return X
    
    def add_noise(self, X):
        for i in range(X.shape[0]):
            noise = self.noises[np.random.choice(np.arange(len(self.noises)))]
            index = np.random.choice(np.arange(X.shape[1] - noise.shape[0]))
            if np.random.choice([1, 0], p=[self.p_noise, 1-self.p_noise]):
                X[i, index:index+noise.shape[0], :] = (X[i, index:index+noise.shape[0], :]).astype(np.float16) + noise
        return X